In [ ]:
- Chúng ta hoàn toàn có thể lấy được Object Response từ Hook (`useCreateProductMutation`)
- Nhưng `newProduct` (response được lấy trực tiếp từ API kết hợp với `unwrap()`) vẫn cần thiết cho các mục đích khác nhau:

# 1. Cú pháp lấy object response từ Hook
- Khi bạn gọi mutation, nó trả về một mảng gồm 2 phần tử:
    1. Hàm kích hoạt (Trigger Function): `createProduct`
    2. Object trạng thái: Chứa `data`, `error`, `isLoading`, `isSuccess`, ...

In [ ]:
const [createProduct, { data, isLoading, error, isSuccess }] = useCreateProductMutation();

- Ở đây, biến `data` chính là response trả về từ API (tương đương `newProduct` bạn lấy được từ `.unwrap()`)

# 2. Tại sao vẫn cần `newProduct` từ `unwrap()`?

- Thắc mắc: "Nếu `data` trong hook có dữ liệu rồi, sao tôi phải dùng `const newProduct = await...unwrap()` làm gì nữa?"

## A. Object từ Hook (`{ data, isLoading }`) -> Dùng để Render UI (Giao diện)
- Object này là dạng Reactive (tự động cập nhật). Nó sinh ra để phục vụ việc hiển thị lên màn hình.
    - Bạn dùng `isLoading` để xoay vòng tròn loading
    - Bạn dùng `error` để hiện dòng chữ đỏ báo lỗi
    - Bạn dùng `isSuccess` để hiện dấu tích xanh

=> Hạn chế: Rất khó dùng biến `data` để xử lý logic tuần tự (như chuyển trang) ngay trong hàm `onSubmit`. Vì lúc hàm `onSubmit` đang chạy, `data` trong hook chưa kịp cập nhật

## B. Biến từ `unwrap()` (`newProduct`) -> Dùng để xử lý luồng Logic
- Biến này là dạng Imperative (mệnh lệnh). Nó sinh ra để phục vụ luồng đi của code trong hàm xử lý sự kiện,
    - Bạn cần ID của sản phẩm ngay lập tức sau khi tạo xong để chuyển trang
    - Bạn cần dữ liệu ngay lập tức để hiện thông báo `toast` cụ thể (ví dụ: Đã tạo sản phẩm A giá 100k)

# 3. Ví dụ minh hoạ sự kết hợp hoàn hảo
Hãy xem code dưới đây, mình dùng CẢ HAI, mỗi thứ một việc:

In [ ]:
const CreateProductForm = () => {
  // 1. Lấy cả hàm trigger VÀ object trạng thái
  const [createProduct, { isLoading, error }] = useCreateProductMutation();
  const navigate = useNavigate();

  const handleCreate = async (formData) => {
    try {
      // --- Dùng unwrap() cho LOGIC ---
      // Mình cần biến 'newProduct' ở đây để lấy ID chuyển trang NGAY LẬP TỨC
      const newProduct = await createProduct(formData).unwrap();
      
      toast.success(`Đã tạo: ${newProduct.name}`);
      navigate(`/products/${newProduct.id}`); // <-- Cần ID ở đây
      
    } catch (err) {
      // Xử lý lỗi logic nếu cần
    }
  };

  return (
    <form onSubmit={handleSubmit(handleCreate)}>
      {/* --- Dùng object từ Hook cho GIAO DIỆN --- */}
      
      {/* Dùng error để hiện lỗi lên màn hình */}
      {error && <div className="error-msg">{error.data.message}</div>}

      <input name="name" />
      
      {/* Dùng isLoading để disable nút bấm */}
      <button type="submit" disabled={isLoading}>
        {isLoading ? "Đang tạo..." : "Tạo sản phẩm"}
      </button>
    </form>
  );
};

Kết luận:
- Nếu bạn chỉ cần **hiển thị** kết quả lên màn hình (ví dụ: hiện thông tin user sau khi update):
- Nếu bạn cần **dùng kết quả đó để làm việc tiếp theo** (redirect, gọi API khác):
    + Dùng `unwrap()`

In [ ]:
- Nếu bạn không dùng `.unwrap()` hay `await` để lấy kết quả ngay tại chỗ.
- Mà bạn cố gắng dùng **Object trả về từ Hook** (`data`, `isSuccess`) để xử lý logic (như chuyển trang)
> Gặp vấn đề: "Độ trễ của React State"
> Để giải quyết, bạn bắt buộc dùng `useEffect`. Điều này làm code trở nên rời rạc và khó kiểm soát

# Ví dụ kịch bản: TẠO SẢN PHẨM XONG THÌ CHUYỂN HƯỚNG TRANG
Cách làm: Dùng `useEffect` (Khi không dùng `unwrap`)
- Vì `isSuccess` và `data` không cập nhật ngay lập tức trong hàm handleCreate, bạn phải sinh ra một "người dám sát" (`useEffect`) để ngồi canh chừng khi nào chúng thay đổi

In [ ]:
import { useEffect } from 'react';

const CreateProductForm = () => {
  // 1. Lấy trạng thái từ Hook
  const [createProduct, { data, isSuccess, isError, error }] = useCreateProductMutation();
  const navigate = useNavigate();

  // 2. Hàm submit chỉ có nhiệm vụ "Kích hoạt"
  const handleCreate = (formData) => {
    createProduct(formData);
    // ❌ KHÔNG THỂ chuyển trang ở đây được!
    // Tại dòng này, biến 'isSuccess' vẫn là false, 'data' vẫn là undefined.
    // Vì React chưa kịp render lại.
  };

  // 3. Phải dùng useEffect để "canh chừng" kết quả
  useEffect(() => {
    if (isSuccess && data) {
      // Khi thấy báo thành công thì mới chuyển trang
      toast.success("Tạo thành công!");
      navigate(`/products/${data.id}`);
    }
  }, [isSuccess, data, navigate]); // Phụ thuộc vào sự thay đổi của isSuccess

  // 4. Lại phải thêm một useEffect khác để bắt lỗi (nếu muốn tách bạch)
  useEffect(() => {
    if (isError) {
      toast.error(error?.data?.message);
    }
  }, [isError, error]);

  return (
    <button onClick={() => handleCreate({ name: 'A' })}>Tạo</button>
  );
};

In [ ]:
# Tại sao cách này "dở" hơn dùng `unwrap()`
- Nếu nhìn vào code trên, bạn sẽ thấy được 3 nỗi đau:
  1. Logic bị xé lẻ:
    - Hành động bấm nút nằm ở dòng 10
    - Hành động chuyển trang nằm ở dòng 17
    > Nhảy mắt lên xuống liên tục để hiểu luồng đi. Trong khi với `unwrap()`, mọi thứ nằm gọn trong 1 hàm

  2. Rủi ro "Vòng lặp vô tận" hoặc "Lỗi logic khi quay lại"
    - Giả sử bạn tạo xong, chuyển sang trang B
    - Người dùng bấm nút "Back" trên trình duyệt để quay lại trang Tạo
    - Lúc này `isSuccess` vẫn đang là `true` (do Redux cache lại trạng thái cũ)
      > `useEffect` lại chạy ngay lập tức --> Nó lại chuyển hướng sang trang B. Người dùng bị kẹt, không thể quay lại form cũ
      > Khắc phục: Lại phải reset thủ công hook (`reset()`) khi component unmount => Phiền
  
  3. Khó xử lý chuỗi hành động:
    - Nếu bạn muốn: Tạo xong -> Lấy ID -> Upload ảnh -> Chuyển trang
    - Dùng `useEffect` thì bạn phải viết một chuỗi các effect phụ thuộc nhau -> rối code

# Tóm lại hình ảnh so sánh:
  - Dùng `unwrap()`: Giống như bạn đi mua hàng:
    + Bạn đứng chờ ở quầy -> trả tiền -> nhận hàng -> rồi đi về (Xử lý đồng bộ liền mạch)
  - Dùng `useEffect()`: Giống như đặt hàng online:
    + Bạn bấm "Submit".
    + Sau đó cứ phải ngồi F5 điện thoại (useEffect) chờ tin nhắn báo "Giao thành công" (`isSuccess = true`) thì mới được đi làm việc khác

## Rủi ro "Vòng lặp vô tận" hoặc "Lỗi logic khi quay lại"

- Hãy tưởng tượng bạn xây dựng chức năng: Cập nhật thông tin người dùng
    1. Người dùng sửa tên, bấm LƯU
    2. Nếu thành công -> Chuyển hướng về trang Trang chủ

- Code dùng `useEffect`

In [ ]:
const UpdateProfile = () => {
  const [updateUser, { isSuccess }] = useUpdateUserMutation();
  const navigate = useNavigate();

  // Logic: Hễ thấy isSuccess = true là chuyển trang
  useEffect(() => {
    if (isSuccess) {
      toast.success("Đã lưu!");
      navigate("/home");
    }
  }, [isSuccess, navigate]);

  return <button onClick={() => updateUser('...') }>Lưu</button>;
};

In [ ]:
# Rủi ro 1: Lỗi logic khi quay lại:

- Chuyện gì xảy ra:
  1. Bạn bấm Lưu -> API chạy xong -> Biến `isSuccess` chuyển từ `false` -> `true`
  2. `useEffect` thấy `true` -> Nó thực hiện đá sang Trang chủ
  3. Nhưng, bạn lại chợt nhớ ra: "Chết, mình quên chưa sửa số điện thoại!"
  4. Bạn bấm nút Back trên trình duyệt để quay lại trang Update

- Thảm hoạ bắt đầu:
  + Khi component được mount lại, nó kết nối với trạng thái của RTK Query
  + Vì chưa `reset()` (clean up), Redux vẫn nhớ trạng thái cuối cùng: `isSuccess = true`
  + Khi component mount lại -> chạy `useEffect` -> Nó kiểm tra: "Ủa, `isSuccess` là `true` => Lập tức đá về trang chủ lần nữa


In [ ]:
// Cách sửa:

const [updateUser, { isSuccess, reset }] = useUpdateUserMutation();

useEffect(() => {
  if (isSuccess) {
    navigate("/home");
  }
  
  // Cleanup function: Reset trạng thái khi rời khỏi component này
  return () => {
    reset(); 
  };
}, [isSuccess, reset, navigate]);

In [ ]:
So sánh:
  - Dùng `unwrap()`: 1 dòng code, logic thẳng tuột, tự động xong việc là xong
  - Dùng `useEffect`: Phải thêm cleanup, phải quản lý deps

# 1. Tại sao `deps` không đổi mà `useEffect` vẫn chạy?
- Thắc mắc: "`isSuccess` vẫn đang là `true` (tức là `deps` trong `useEffect` không đổi) thì sao nó vẫn chạy vào nhỉ?
- Câu trả lời: Là do cơ chế Mounting (khi Component được sinh ra)
- Quy tắc `useEffect`:
  + `useEffect`: LUÔN LUÔN chạy ít nhất 1 lần ngay sau khi Component được Mount (lần render đầu tiên), bất kể dependency có thay đổi hay không
- Dependency  [isSuccess] chỉ có tác dụng ngăn chặn nó chạy lại ở các lần Update (re-render) tiếp theo thôi.
- Diễn biến thực tế:
  1. Nhấn Back -> Component được tạo mới (được mount lại)
  2. Render giao diện
  3. Chạy `useEffect` lần đầu tiên
  4. Bên trong `useEffect` là `true` --> chuyển trang

# Tại sao khi Mount là `isSuccess = true` (lẽ ra phải là `false`)
- Thắc mắc: Và nếu mount lại thế thì ban đầu `isSuccess` là `false` chứ nhỉ?
- Câu trả lời: Khác biệt giữa Local State (`useState`) & Global State (RTK Query)
  + Nếu bạn dùng `useState`: Đúng! Khi Component unmount (bị huỷ) và mount lại, `useState` sẽ bị reset về giá trị khởi tạo (`false`)
  + Nhưng bạn đang dùng RTK Query (Redux): Dữ liệu KHÔNG nằm trong Component, nó nằm ở Redux Store (Global Store)

- Khi bạn dùng `useMutate`, RTK Query lưu trạng thái của mutation đó vào Redux Store
1. Lần 1 (Lúc bạn bấm Lưu)
  + Component gọi API
  + Redux Store cập nhật: `UpdateMutation: { status: 'fulfilled', isSuccess: true }.`
  + Chuyển trang sang Home -> Component trước đó bị huỷ (Unmount)
  > QUAN TRỌNG: Mặc định, trạng thái trong Redux Store **VẪN CÒN ĐÓ**, nó chưa bị xoá ngay (trừ khi bạn reset thủ công hoặc store tự dọn dẹp sau một khoảng thời gian)

2. Lần 2 (Lúc bạn nhấn Back quay lại)
  + Component được mount lại
  + Hook `useUpdateUserMutation()` chạy lại. Nó kết nối vào Redux Store để xem tình hình
  + Nó thấy trong Store vẫn còn lưu kết quả cũ: "À, cái mutation này lần trước chạy thành công rồi nè "`isSuccess = true`"
  + => Hook trả về ngay lập tức isSuccess: true cho Component mới này.
  + Hệ quả: Component vừa mới sinh ra (Mount) -> hook lấy ngay giá trị `true` từ kho lưu trữ (Store)
  => -> `useEffect` chạy lần đầu -> Thấy `true` -> Chuyển trang luôn. 

  # TÓM TẮT TRỰC QUAN:
  - Để dễ hình dung, hãy tưởng tượng:
    + Component: Là một nhân viên tại quầy
    + Redux Store: là cái bảng thông báo treo trên tường
    + `useEffect`: Là quy trình làm việc:
      * Sáng đến công ty (Mount)
      * Nhìn lên bảng -> nếu thấy ghi chữ "XONG" -> đi về nhà

  - Kịch bản lỗi:
    1. Hôm qua nhân viên A làm xong việc, ghi chữ  "Xong" lên bảng rồi đi về
    2. Sáng hôm sau (Nhấn Back), nhân viên A (hoặc nhân viên mới) đến quầy
    3. Việc đầu tiên anh ta làm (Mounting Effect): Nhìn lên bảng
    4. Anh ta thấy chữ "XONG" vẫn còn đó (do chưa ai xoá)
    5. Theo quy trình, anh ta .. đi về nhà

  - Cách khắc phục: Nếu vẫn muốn dùng `useEffect`:
  - Bạn phải "xoá bảng" khi nhân viên đi về:

In [ ]:
useEffect(() => {
    // Logic chuyển trang
    if (isSuccess) navigate('/home');

    // Cleanup function: Chạy khi component Unmount (bị hủy)
    return () => {
        reset(); // <--- Xóa bảng (Reset isSuccess về false)
    }
}, [isSuccess, reset]);

// Hàm reset() được nằm trong object kết quả được trả về khi gọi hook useMutation

In [ ]:
## Vấn đề "Quay lại" (Back Button) đối với useQuery

- Thắc mắc: "Nếu là Query (lấy dữ liệu) khi nhấn Back quay lại, nó hoạt động thế nào? Có bị dính trạng thái cũ không?"
- Câu trả lời: Có dính trạng thái cũ (Cache) => TÍNH NĂNG
- Kịch bản:
  1. Bạn vào trang Danh sách sản phẩm
    + `useGetProductsQuery()` chạy -> Gọi API -> Lưu vào Cache -> Hiển thị list.
  2. Bạn bấm vào một sản phẩm để sang trang chi tiết
  3. Bạn bấm nút Back để quay lại trang Danh sách

- Diễn biến lúc quay lại (Mount lại component)
  1. Component mount: `useGetProductsQuery()` được gọi lại.
  2. Kiểm tra Cache (Hỏi Store): Nó hỏi: "Store ơi, có dữ liệu danh sách sản phẩm chưa?"
  3. Store trả lời: "Có rồi nè, nãy mới tải xong, còn 'tươi' (fresh) lắm
  4. Kết quả:
    + `data`: Trả về mảng sản phẩm NGAY LẬP TỨC
    + `isLoading`: False (Không quay vòng tròn nữa)
    + `isSuccess`: True


## Tại sao điều này TỐT với Query (nhưng xấu với Mutation)?
- Trải nghiệm người dùng UX: Người dùng thấy danh sách hiện ra ngay lập tức, không phải chờ loading lại từ đầu. Đây chính là sức mạnh của Caching.
- Không có "Bẫy logic":
  + Với **Mutation** (như Tạo mới/Update) hành động là "**Thay đổi**". Nếu nó tự chạy lại hoặc báo thành công giả => lỡ tay chuyển trang SAI mục đích
  + Với **Query** (như Lấy danh sách), hành động là "**Hiển thị**"
      * Việc nó báo thành công ngay lập tức là điều tuyệt vời => Giao diện hiện lên luôn
      * Bạn đâu có viết kiểu:
        ❌ Nếu lấy được danh sách sản phẩm thì chuyển trang khác đâu
        ✅ Nếu có data thì render ra màn hình thôi
